# 🧠 Modelo de Regressão para Previsão de Preços de Veículos


## 📌 2. Data Understanding

**Objetivo**: entender o conteúdo, a estrutura e a qualidade dos dados.

**Atividades:**
- Analisa a qualidade inicial dos dados
- Verifica valores ausentes, tipos de dados, distribuições
- Identifica valores extremos, inconsistências, duplicatas
- Verifica a estrutura dos dados
- Realiza estatísticas descritivas iniciais (describe, value_counts)



In [2]:
# Carrega Bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import normaltest

%matplotlib inline

# Setup para mostrar todas as colunas do dataframe
pd.set_option('display.max_columns', None)

In [3]:
# Carrega Dataset

colls = ["symboling","normalized-losses","make","fuel-type","aspiration", "num-of-doors","body-style",
         "drive-wheels","engine-location","wheel-base", "length","width","height","curb-weight","engine-type",
         "num-of-cylinders", "engine-size","fuel-system","bore","stroke","compression-ratio","horsepower",
         "peak-rpm","city-mpg","highway-mpg","price"]


file_path = "https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data"
df = pd.read_csv(file_path, names=colls)

# Visualiza as 5 primeiras linhas
df.head()


,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450


In [4]:
# Verifica o número de colunas e linhas do datafrme
total_raw_rows = df.shape[0]
total_raw_cols = df.shape[1]
print("Nº de Linhas:",total_raw_rows)
print("N° de Colunas:",total_raw_cols)

Nº de Linhas: 205
N° de Colunas: 26


In [5]:
# Verifica o tipo de dados das colunas
print("Tipo de Dados:")
df.dtypes

Tipo de Dados:


symboling              int64
normalized-losses     object
make                  object
fuel-type             object
aspiration            object
num-of-doors          object
body-style            object
drive-wheels          object
engine-location       object
wheel-base           float64
length               float64
width                float64
height               float64
curb-weight            int64
engine-type           object
num-of-cylinders      object
engine-size            int64
fuel-system           object
bore                  object
stroke                object
compression-ratio    float64
horsepower            object
peak-rpm              object
city-mpg               int64
highway-mpg            int64
price                 object
dtype: object

In [6]:
# Verifica valores nulos
print("Valores Nulos:")
print(df.isnull().sum())

# Mostra valores nulos
#df[df.isnull()]

Valores Nulos:
symboling            0
normalized-losses    0
make                 0
fuel-type            0
aspiration           0
num-of-doors         0
body-style           0
drive-wheels         0
engine-location      0
wheel-base           0
length               0
width                0
height               0
curb-weight          0
engine-type          0
num-of-cylinders     0
engine-size          0
fuel-system          0
bore                 0
stroke               0
compression-ratio    0
horsepower           0
peak-rpm             0
city-mpg             0
highway-mpg          0
price                0
dtype: int64


In [10]:
# Verifica valores únicos das Variáveis Categóricas

categorical_features = ['symboling', 'make',  'fuel-type', 'aspiration', 'num-of-doors', 'body-style', 'drive-wheels',
                                'engine-type','num-of-cylinders', 'fuel-system']

for col in categorical_features:
    print(f"\n{col} = {df[col].unique()}")




symboling = [ 3  1  2  0 -1 -2]

make = ['alfa-romero' 'audi' 'bmw' 'chevrolet' 'dodge' 'honda' 'isuzu' 'jaguar'
 'mazda' 'mercedes-benz' 'mercury' 'mitsubishi' 'nissan' 'peugot'
 'plymouth' 'porsche' 'renault' 'saab' 'subaru' 'toyota' 'volkswagen'
 'volvo']

fuel-type = ['gas' 'diesel']

aspiration = ['std' 'turbo']

num-of-doors = ['two' 'four' '?']

body-style = ['convertible' 'hatchback' 'sedan' 'wagon' 'hardtop']

drive-wheels = ['rwd' 'fwd' '4wd']

engine-type = ['dohc' 'ohcv' 'ohc' 'l' 'rotor' 'ohcf' 'dohcv']

num-of-cylinders = ['four' 'six' 'five' 'three' 'twelve' 'two' 'eight']

fuel-system = ['mpfi' '2bbl' 'mfi' '1bbl' 'spfi' '4bbl' 'idi' 'spdi']


In [ ]:
# Verificar valores Inconsistentes '?' que precisam ser substituídos
print("Contagem de valores '?' por coluna:")
print((df == '?').sum())

In [ ]:
# Verifica a presença de dados duplicados
df.duplicated().sum()

# Mostra os dados duplicados, se houver
df[df.duplicated()]

In [ ]:
# Trata inconsistência dos dados para analisar a variável `price`
df['price'] = df['price'].replace('?', np.nan)
df.dropna(subset=['price'], axis=0, inplace=True)
# Converte cooluna para formato numérico
df["price"] = pd.to_numeric(df["price"])

print("Resumo Estatístico - Variáveis Numéricas:")
df.describe()

In [ ]:
print("Resumo Estatístico - Variáveis Categóricas:")
df.describe(include=['object'])

In [ ]:
# Análise da distribuição da variável target (price)
plt.figure(figsize=(10, 6))
sns.histplot(df['price'], bins=30, kde=True)
plt.title('Distribuição dos Preços dos Carros')
plt.xlabel('Preço')
plt.ylabel('Contagem')
plt.show()


print("Mediana", df['price'].median())
print("Media", df['price'].mean())
print("Distribuição Assimetrica á Direita: moda < media < mediana ")

In [ ]:
signif = 0.05
stat_test, p_valor = normaltest(df['price'])
print(stat_test)
print(p_valor)

# True - ou seja rejeita a Hipotese Nula, nos diz que a coluna `price` não faz parte de uma distribuição normal
if p_valor <= signif:
    print("Hipótese Nula Rejeitada - Distribuição não é normal!!")
else:
    print("Distribuição é normal!!")

In [ ]:
# Verifica Presença de Outliers

Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_outliers = df[((df['price'] < lower_bound) | (df['price'] > upper_bound))]

print("Total de Outliers Identificados: ", df_outliers.shape[0])

ax = sns.boxplot(x='price', data=df)

ax.figure.set_size_inches(12, 4)
ax.set_title('Preço', fontsize=18)
ax.set_xlabel('USD', fontsize=14)
ax

In [ ]:
# Verifica a distribuição dos Dados por fabricante
ax = sns.boxplot(x='price', y='make', data=df)

ax.figure.set_size_inches(12, 4)
ax.set_title('Preço por Fabricante', fontsize=18)
ax.set_xlabel('Fabricante', fontsize=14)
ax

# Analisando a dispersão por fabricante, vemos que os veículos de marca = ['bmw, 'mercedez-benz', `porsche`] veículos de luxo possui uma maior discrepância

In [ ]:
# Plota BoxPlot com preços > 30k por fabricante
ax = sns.boxplot(x='price', y='make', data=df.query('price > 30000') )

ax.figure.set_size_inches(12, 4)
ax.set_title('Preço por Fabricante', fontsize=18)
ax.set_xlabel('Fabricante', fontsize=14)
ax

# Filtrando os preços > 30k  fica mais evidente que os fabricantes bmw e mercedez possuem uma maior discrepância 

### 📋 Initial Observations

#### Issues Qualidade dos Dados:

| Nº Item | Item                                                   | Status  | Comentários                                                 |
|---------|--------------------------------------------------------|---------|-------------------------------------------------------------|
| 01      | Valores ausentes (Missing Data)                        | OK      | não identificado                                            |
| 02      | Duplicidade de registros                               | OK      | não identificado                                            |
| 03      | Inconsistência de formato                              | OK      | não identificado                                            |
| 04      | Erros de digitação                                     | OK      | não identificado                                            |
| 05      | Valores fora do intervalo esperado (Outliers)          | not OK  | identificado 14 registros com preços acima de 30k           |
| 06      | Tipos de dados incorretos                              | Not OK  | alterar formato das colunas categórica `object`             |
| 07      | Dados inconsistentes entre colunas                     | Not OK  | Sete colunas com valores inconsistentes "?"                 |
| 08      | Alta cardinalidade em variáveis categóricas            | Ok      | não identificado                                            |
| 09      | Dados desatualizados                                   | NA      | não aplicável                                               |
| 10      | Dados incompletos (sem cobertura em algumas regiões)   | Ok      | não identificado                                            |


#### Distribuição dos Dados
- Presença de outliers na variável `price` com 14 registros acima da média, verificado que os fabricantes `bmw`, `mercedez-bens`, `porsche` possui uma maior discrepância com valores acima de 30k
- A distribuição da variável target `price` é Assimétrica (Skewness) á Direita (Moda < Media < Mediana);
- O teste de normalidade rejeitou a hipótese nula, logo a distribuição não é normal (p-value < significancia_alpha).

